# Proceso ETL 

In [31]:
import glob
import pandas as pd
from models.song import Song
from models.artist import Artist
from models.songPlay import SongPlay
from models.time import Time
from models.user import User
import numpy as np
from bd import Base,session,engine
from sqlalchemy import exc

# Lectura de Archivos
* La función get_nameFiles obtiene los nombres de todos los archivos en la ruta enviada

In [32]:
#Function get all files names from path 
def get_nameFiles(path):
    files = [f for f in glob.glob(path + "**/*.json", recursive=True)]
    return files

* La función files_To_DataFrame concentra en un DataFrame el contenido de todos los archivos

In [33]:
#Function 
def files_To_DataFrame(Filejson):
    df_full= pd.DataFrame()
    for file in Filejson:
        #Read file
        df = pd.read_json(file,lines=True,encoding='utf-8') 
        #Data Frame
        df_full = df_full.append(df)
    return df_full

## Se procesan los archivos 'data/song_data/'

Se obtiene los nombres y ruta de todos los archivos que se encuentran en 'data/song_data/'

In [34]:
#Se obtienen los nombre de todos los archivos los archivos
files = get_nameFiles('data/song_data/')
files[0]

'data/song_data/A/B/B/TRABBVJ128F92F7EAA.json'

Se obtiene el contenido de cada archivo y se guardan en un dataFrame

In [35]:
#Se crea un dataframe con la información de todos los archivos
df_song_data = files_To_DataFrame(files)
df_song_data.head(n=1)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AREDL271187FB40F44,NaN,NaN,,Soul Mekanik,SOPEGZN12AB0181B3D,Get Your Head Stuck On Your Neck,45.66159,0


# Tablas de hechos y de dimensiones
En esta parte, se obtienen y limpian los datos para las tablas:
* **song** (dimensiones)
* **artist** (dimensiones)
* **time_format** (dimensiones)
* **user** (dimensiones)
* **song_Play** (hechos)

1.Se obtienen los datos para la tabla **song**. Se eliminan los duplicados

In [36]:
#Se obtienen los datos para la tabla SONG
df_song = df_song_data[['song_id','artist_id', 'title','year', 'duration']].dropna(subset=['song_id']).drop_duplicates(['song_id'])

In [37]:
df_song.head(n=1)

,song_id,artist_id,title,year,duration
0,SOPEGZN12AB0181B3D,AREDL271187FB40F44,Get Your Head Stuck On Your Neck,0,45.66159


2. Se obtiene los para la tabla **artist**.Se eliminan los duplicados

In [38]:
#Se obtienen los datos para la tabla ARTIST
df_artist = df_song_data[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].dropna(subset=['artist_id']).drop_duplicates(['artist_id'])
df_artist = df_artist.rename(columns={'artist_name': 'name', 'artist_location': 'location', 'artist_latitude': 'latitude', 'artist_longitude': 'longitude'})

In [39]:
df_artist.head(n=1)

,artist_id,name,location,latitude,longitude
0,AREDL271187FB40F44,Soul Mekanik,,NaN,NaN


## Se procesan los archivos 'data/log_data/'

Se obtiene los nombres y ruta de todos los archivos que se encuentran en 'data/song_data/'

In [40]:
#Se obtienen los nombre de todos los archivos los archivos
files = get_nameFiles('data/log_data/')

In [41]:
files[0]

'data/log_data/2018/11/2018-11-22-events.json'

Se obtiene el contenido de cada archivo y se guardan en un dataFrame

In [42]:
#Se crea un dataframe con la información de todos los archivos
df_long_data = files_To_DataFrame(files)
df_long_data = df_long_data[df_long_data['page'] == 'NextSong']

In [43]:
df_long_data.head(n=1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Dee Dee Bridgewater,Logged In,Lily,F,38,Koch,318.64118,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,La Vie En Rose,200,1542845032796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


3. Se obtiene los para la tabla **time_format**.Se eliminan los duplicados

In [44]:
#Se obtienen los datos para la tabla TIME_FORMAT
ts = pd.to_datetime(df_long_data['ts'], unit='ms')
timeFormat = (df_long_data['ts'].values, ts.dt.hour.values, ts.dt.day.values, ts.dt.week.values, ts.dt.month.values, ts.dt.year.values, ts.dt.weekday.values)
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']
df_time_format= pd.DataFrame(data=list(zip(*timeFormat)), columns=column_labels)
df_time_format = df_time_format.drop_duplicates(['start_time'])

In [45]:
df_time_format.head(n=1)

,start_time,hour,day,week,month,year,weekday
0,1542845032796,0,22,47,11,2018,3


4. Se obtiene los para la tabla **user**.Se eliminan los duplicados

In [46]:
#Se obtiene los datos para la tabla USER
convert_dict = {'user_id': int } 
df_user = df_long_data[['userId', 'firstName', 'lastName', 'gender', 'level']].rename(columns={'userId': 'user_id'}).replace('', np.nan).dropna(subset=['user_id'])
df_user = df_user.astype(convert_dict)
df_user = df_user.drop_duplicates(['user_id'])

In [47]:
df_user.head(n=1)

,user_id,firstName,lastName,gender,level
0,15,Lily,Koch,F,paid


5. Se obtiene los para la tabla **song_play**.Se hace un JOIN para obtener los identificadores user_id y artist_id

In [48]:
df_song_data_aux = df_song_data[['artist_id','song_id', 'title','artist_name']].rename(columns={'title': 'song', 'artist_name': 'artist'})
df_long_data_aux = df_long_data[['userId', 'sessionId','userAgent','level','location','ts','artist','song']]

In [49]:
df_song_play_join = pd.merge(df_long_data_aux, df_song_data_aux, on=['song','artist'], how='left')
df_song_play_join = df_song_play_join.rename(columns={'ts': 'start_time', 'sessionId': 'session_id','userAgent': 'user_agent','userId': 'user_id'}).replace('',0)

In [50]:
df_song_play_join['songplay_id'] = df_song_play_join.index

In [51]:
df_song_play_join = df_song_play_join.drop(['song', 'artist'], axis=1)


In [52]:
df_song_play_join.head(n=1)

,user_id,session_id,user_agent,level,location,start_time,artist_id,song_id,songplay_id
0,15,818,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",paid,"Chicago-Naperville-Elgin, IL-IN-WI",1542845032796,NaN,NaN,0


# Se crean los objetos para ser guardados en BD
* En la carpeta models, se encuentran las clases que corresponde al mapeo de las tablas 
* A partir del dataFrame se crea una lista de objetos

In [53]:
listObjectSong       = [Song(**kwargs) for kwargs in df_song.to_dict(orient='records')]
listObjectArtist     = [Artist(**kwargs) for kwargs in df_artist.to_dict(orient='records')]
listObjectTime       = [Time(**kwargs) for kwargs in df_time_format.to_dict(orient='records')]
listObjectUser       = [User(**kwargs) for kwargs in df_user.to_dict(orient='records')]
listObjectSongPlay   = [SongPlay(**kwargs) for kwargs in df_song_play_join.to_dict(orient='records')]

In [54]:
listObjectSong[0]

<Song(song_id='SOPEGZN12AB0181B3D', title='Get Your Head Stuck On Your Neck', artist_id='AREDL271187FB40F44', year=0, duration=45.66159)>

# Se  crean las tablas

Por medio de la librería **SQLAlchemy** se eliminan las tablas si es que existen y se crean nuevamente.

In [55]:
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

Se guardan los registros con bulk_save_objets en PostgreSQL

In [56]:
try:
    session.bulk_save_objects(listObjectSong)
    session.bulk_save_objects(listObjectArtist)
    session.bulk_save_objects(listObjectTime)
    session.bulk_save_objects(listObjectUser)
    session.bulk_save_objects(listObjectSongPlay)
    session.commit()
    session.close()
except exc.SQLAlchemyError as ex:
    session.close()
    print('Exeption:', ex)